In [1]:
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings("ignore")
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from dataloaders import data_set,data_dict
import pywt
import numpy as np

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()    

# ================ change the path to dataset ! 
args.root_path =  r"E:\datasets\RWhar_Dataset"
# ================ If using spectrogram as input, modify the path where spectrogram is stored
args.freq_save_path = r"E:\datasets\Freq_data"
# ================ 
args.data_name = "rw"

args.difference = True 
args.sampling_freq =   50
args.train_vali_quote = 0.95
# for this dataset the window size is 128
args.windowsize = int(2.56 * args.sampling_freq)




args.drop_long = False

args.datanorm_type = "standardization" # None ,"standardization", "minmax"

args.wavename = "morl"
# if you want to use raw time series as input, set model_type as time
# if you want to use spectrogram as input, set model_type as freq
# if you want to use both as inputs, set model_type as cross
args.model_type = "time"

# if you want to do Leave one out cross validation, set exp_mode as "LOCV"
# if you want to do the given train test experiment, set exp_mode as "Given"
# if you want to do Semi non overlapping experiment, set exp_mode as "SOCV"
# if you want to do Full non overlapping experiment, set exp_mode as "FOCV"
args.exp_mode = "SOCV"

if args.exp_mode == "FOCV":
    args.displacement =  int(1 * args.windowsize) 
else:
    args.displacement =  int(0.5 * args.windowsize)     


args.batch_size = 32
args.shuffle = True
args.drop_last = False
dataset = data_dict[args.data_name](args)




 ----------------------- load all the data -------------------
----------------------- Get the Sliding Window -------------------
The orginal class weights are :  [1.2491 1.0778 5.9476 0.8327 0.7645 0.8341 0.8437 0.8261]


In [3]:
dataset.data_x

,sub_id,chest_acc_x,chest_acc_y,chest_acc_z,forearm_acc_x,forearm_acc_y,forearm_acc_z,head_acc_x,head_acc_y,head_acc_z,...,diff_thigh_acc_x,diff_thigh_acc_y,diff_thigh_acc_z,diff_upperarm_acc_x,diff_upperarm_acc_y,diff_upperarm_acc_z,diff_waist_acc_x,diff_waist_acc_y,diff_waist_acc_z,sub
0,1_0,5.589264,8.054693,0.869095,-9.659440,-1.678860,0.864563,-4.313154,8.675988,0.289100,...,-0.007632,0.004040,-0.006434,-0.037110,0.024541,-0.004788,0.009577,0.046088,-0.108936,1
1,1_0,5.580884,8.060080,0.908001,-9.653410,-1.675230,0.853531,-4.326921,8.726865,0.279523,...,-0.007632,0.004040,-0.006434,-0.037110,0.024541,-0.004788,0.009577,0.046088,-0.108936,1
2,1_0,5.588067,8.033744,0.884657,-9.655440,-1.697300,0.842896,-4.317344,8.771756,0.241216,...,-0.007632,0.004040,-0.006434,-0.049081,0.024541,0.003591,0.010175,0.005387,0.001796,1
3,1_0,5.583877,8.060678,0.875679,-9.659870,-1.689700,0.825745,-4.347870,8.815451,0.199916,...,-0.005088,-0.002094,0.014665,-0.002394,-0.046089,-0.041300,0.004190,-0.084396,0.075417,1
4,1_0,5.596446,8.047510,0.858920,-9.674440,-1.709010,0.855850,-4.347870,8.743625,0.240617,...,-0.005387,0.001197,0.002394,0.102951,0.047884,0.043096,0.018556,-0.021548,0.077213,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214838,9_7,-0.530914,9.511565,1.781885,-9.589110,-2.252350,0.754479,-9.104550,3.413533,-0.504578,...,0.031723,0.035913,-0.066439,0.026336,-0.005985,0.025139,0.010375,-0.007432,0.023892,9
3214839,9_7,-0.574010,9.560646,1.751957,-9.598888,-2.219933,0.730026,-9.215881,3.350685,-0.489016,...,0.041300,0.043694,0.017358,-0.008380,0.022744,-0.002993,0.012569,-0.007881,-0.004489,9
3214840,9_7,-0.606332,9.585785,1.720832,-9.608665,-2.187515,0.705574,-9.408614,3.235165,-0.467468,...,0.043096,-0.036511,0.037709,-0.048483,-0.064642,0.036512,0.012569,-0.007881,-0.004489,9
3214841,9_7,-0.608726,9.590573,1.709460,-9.623212,-2.182993,0.763356,-9.228451,3.208230,-0.407613,...,0.032920,-0.013168,-0.056862,-0.029329,-0.002992,0.014964,-0.043096,-0.011722,-0.008978,9


In [4]:
dataset.data_x.isna().sum()

sub_id                 0
chest_acc_x            0
chest_acc_y            0
chest_acc_z            0
forearm_acc_x          0
forearm_acc_y          0
forearm_acc_z          0
head_acc_x             0
head_acc_y             0
head_acc_z             0
shin_acc_x             0
shin_acc_y             0
shin_acc_z             0
thigh_acc_x            0
thigh_acc_y            0
thigh_acc_z            0
upperarm_acc_x         0
upperarm_acc_y         0
upperarm_acc_z         0
waist_acc_x            0
waist_acc_y            0
waist_acc_z            0
diff_chest_acc_x       0
diff_chest_acc_y       0
diff_chest_acc_z       0
diff_forearm_acc_x     0
diff_forearm_acc_y     0
diff_forearm_acc_z     0
diff_head_acc_x        0
diff_head_acc_y        0
diff_head_acc_z        0
diff_shin_acc_x        0
diff_shin_acc_y        0
diff_shin_acc_z        0
diff_thigh_acc_x       0
diff_thigh_acc_y       0
diff_thigh_acc_z       0
diff_upperarm_acc_x    0
diff_upperarm_acc_y    0
diff_upperarm_acc_z    0


In [5]:
print("================ {} Mode ====================".format(dataset.exp_mode))
print("================ {} CV ======================".format(dataset.num_of_cv))
for i in range(dataset.num_of_cv):
    dataset.update_train_val_test_keys()
    train_data  = data_set(args,dataset,"train")
    test_data  = data_set(args,dataset,"test")
    vali_data  = data_set(args,dataset,"vali")
    
    
        # form the dataloader
    train_data_loader = DataLoader(train_data,  
                                    batch_size   =  args.batch_size,
                                    shuffle      =  args.shuffle,
                                    num_workers  =  0,
                                    drop_last    =  args.drop_last)

    vali_data_loader = DataLoader(vali_data,  
                                    batch_size   =  args.batch_size,
                                    shuffle      =  args.shuffle,
                                    num_workers  =  0,
                                    drop_last    =  args.drop_last)

    test_data_loader = DataLoader(test_data,  
                                    batch_size   =  args.batch_size,
                                    shuffle      =  args.shuffle,
                                    num_workers  =  0,
                                    drop_last    =  args.drop_last)
    


================ SOCV Mode ====================
================ 5 CV ======================
Overlapping random Experiment : The 0 Part as the test
The class weights are :  [1.2403 1.0839 5.6744 0.832  0.7688 0.8316 0.8439 0.83  ]
Train data number :  38041
The number of classes is :  8
The input_length  is :  128
The channel_in is :  42
Test data number :  10011
Validation data number :  2003
Overlapping random Experiment : The 1 Part as the test
The class weights are :  [1.2451 1.078  6.0652 0.8325 0.7622 0.8389 0.8439 0.8237]
Train data number :  38041
The number of classes is :  8
The input_length  is :  128
The channel_in is :  42
Test data number :  10011
Validation data number :  2003
Overlapping random Experiment : The 2 Part as the test
The class weights are :  [1.255  1.0778 6.0885 0.8431 0.7598 0.8319 0.836  0.826 ]
Train data number :  38041
The number of classes is :  8
The input_length  is :  128
The channel_in is :  42
Test data number :  10011
Validation data number :  